In [1]:
%%HTML
<link rel="stylesheet" type="text/css" href="css/custom.css">

In [ ]:
import pandas as pd
import numpy as np

# Time Series Analysis
![footer_logo](images/logo.png)

## Goal
The goal of this notebook is to familiarise ourselves with working with time series data with Pandas, the data manipulation and analysis package for Python. In order to achieve this, we will get introduced to a real time series dataset and learn some fundamental analysis techniques. 

## Program
1. [Time Utilities in Pandas](#timeutil)
2. [Reading in Time Series Data](#read)
3. [Time-based Manipulations](#mani)
4. [Rolling and Smoothing](#roll)
5. [Summary](#sum)


<a id='timeutil'></a>

## 1. Time Utilities in Pandas 
![footer_logo](images/clock.jpeg)
There are various ways to work with time series data. Most notably, Pandas library has some *amazing* utilities for dealing with timestamp data. In this notebook we will demonstrate some of them, while introducing you to time series analysis.

### Timestamps in Pandas

Pandas can create datetime data from strings formated as `'yyyy-mm-ddThh:mm:ss:ms'` using `pd.Timestamp()`. The date units are years (‘Y’), months (‘M’), weeks (‘W’), and days (‘D’), while the time units are hours (‘h’) in 24 hour format, minutes (‘m’), seconds (‘s’), milliseconds (‘ms’). Note that time units are combined with date units using `'T'

In [ ]:
date = pd.Timestamp('2020-09-30T18:34:56')
print(f'the date is {date.date()} and the time is {date.time()}')
date

Pandas Timestamps support a wide range of [operations](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Timestamp.html) such as finding differences in time between timestamps or getting various parameters about the date like the number of days in the correspondent month

In [ ]:
pd.Timestamp('2020-02-20T18:34:56') - pd.Timestamp('2020-02-18T18:24:32')

In [ ]:
pd.Timestamp('2020-02-20T18:34:56').daysinmonth

Often timestamps are recorded in Unix Time (the number of seconds elapsed since midnight 1 January 1970 UTC +0 time zone). It can be converted to Pandas Timestamp using the
`unit='s'` option. 

In addition, a specific [time zone](https://pvlib-python.readthedocs.io/en/stable/timetimezones.html) can also be added to a Timestamp using the `tz='time_zone_name'` option:

In [ ]:
pd.Timestamp(1513393355, unit='s', tz='US/Pacific')

Timestamps can be converted to other time zones using `.tz_convert('tz_name')`:

In [ ]:
pd.Timestamp('2020-02-20T18:34:56', tz='US/Pacific').tz_convert('Europe/Paris')

### Formatting Timestamps 

Pandas can recognize timestamps written in various common formats with `pd.to_datetime`, even if the data contains a mix of formats:

In [ ]:
pd.to_datetime(['2020-03-27','April 1, 2020', '25/05/2020'])

*`DatetimeIndex` is an array of Timestamps, which also contains metadata about it*

Pandas will try to recognize the right format in a sequence of timestamps, though in some cases, such as US vs European dates, it may read data not as  you may have intended:

In [ ]:
pd.to_datetime(['02/01/2020','03/01/2020'])

To avoid such confusion, Timestamps can be explicitly formated with the `format` option. We recommend to always do this, rather than relying on Pandas figuring it out right for you. This way you can easily deal with even quite unconventional formats:

In [ ]:
pd.to_datetime('02__2020__01', format='%m__%Y__%d')

Another good thing about fixed formatting - we want to know if single dates are unexpectedly in a wrong format. With a fixed formatting we will get an error raised for each such case rather than relying on Pandas figuring it out for us.

<a id='read'></a>

## 2. Reading in Time Series Data
![footer_logo](https://www.gtreview.com/wp-content/uploads/2019/07/Sustainability-Feature-617x413.jpg) 
Throughout the this notebook we will use the *household power consumption* dataset. It comes from [UCI ML repo](https://archive.ics.uci.edu/ml/datasets/individual+household+electric+power+consumption) and contains detailed power consumption time series data of a single household in Paris between 2006 and 2010.

In [ ]:
power = pd.read_csv('data/household_power_consumption.csv')
power.head()

Typically timestamp data is contained in a separate column of standard strings. However, we want to make it machine-readable. In order to achieve this, we can set `parse_dates` with the list of columns to be converted to Pandas timestamp when reading the data with `pd.read_csv`. This automatically identifies the format of the dates. 

However, explicit formatting may be a good idea if we suspect possible errors and non-standard formats in some of the rows. This will ensure that we get an error message if any value does not match our specified format.

In [ ]:
power = pd.read_csv('data/household_power_consumption.csv', parse_dates=['ts'])
power.head()

In [ ]:
power['ts'][0]

For most time-series analysis functionality, we benefit from not only parsing the dates correctly, but also setting the dates as the index in the Pandas DataFrame. 

Let's illustrate the data without the datetime as index: 

In [ ]:
power.plot(figsize=(16,4));

In [ ]:
power = pd.read_csv('data/household_power_consumption.csv', parse_dates=['ts'], index_col='ts')
power.head()

In [ ]:
power.index[0]

The way we access our timestamp values now changes a bit, because `ts` is no longer a regular column, but this does provide us with an advantage with plotting:

In [ ]:
power.plot(figsize=(16,4));

<a id='mani'></a>

## 3. Time-based manipulations

### Easy Aggregations: Resample 

Another advantage of the datetime-index approach is that it provides us with some functionality for easy time-based aggregations. One such aggregation is `resample`. 

For example, we can easily calculate the _mean_ per _year_ by running:

In [ ]:
power.resample('Y').mean()

You can also run the same aggregation per month `M`, week `W`, day `D` or quarter `Q`. Custom aggregation periods are also possible, for example per 4 weeks `4W` or per 3 months `3M`. See [here] for a more comprehensive list of offsets, that can be as specific as _'Business Month Begin'_.

[here]: https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects

If the index is a timestamp that also includes times, then you can also aggregate per hour. The script below demonstrates this by calculating the mean per 3 hours.

In [ ]:
power.resample('3H').mean().head()

Or apply your own custom calculation, for example the spread per month: 

In [ ]:
power.resample('M').apply(lambda x: np.max(x) - np.min(x)).head()

Finally, it also allows us to create a daily data frame - precision in seconds is a little too much granularity.

In [ ]:
power_daily = power.resample('D').sum()

### Time Based Features 

Any column in Pandas that is of dtype `datetime` has a module attached that can be used to perform vectorised datetime operations. This is very similar to the `.str` module attached to string columns. It is a good thing to explore since the alternative is non-vectorised and much slower.

Below is an example of getting the `weekofyear`. Feel free to explore [other](https://pandas.pydata.org/pandas-docs/stable/reference/series.html#api-series-dt) properties and methods.

In [ ]:
(
    power_daily
    .assign(weekofyear = lambda df: df.index.weekofyear)
    .head()
)

### BONUS: Shifting

It can be often useful to shift some variables forward or backwards in time. This can for example help us create variables with lagged values or calculate differences in values between time steps. This can be done using Panda's `shift()` method, which can shift values by a given number of periods (positive or negative). The example below uses this method to create a new variable for electricity consumption during the previous day:

In [ ]:
(
    power_daily
    .assign(consumption_yesterday = lambda df: df.consumption.shift(1))
    .assign(consumption_increase = lambda df: df.consumption - df.consumption_yesterday)
    .head()
)

<a id='roll'></a>

## 4. Rolling and Smoothing

Let us have a closer look at the power consumption patterns during a single year. The simplest way to plot timestamp data dynamics in Pandas is using `plot()`, which by default plots a linear plot over time:

In [ ]:
power_daily2008 = power_daily['2007-07-01':'2008-09-01']

power_daily2008.plot(figsize=(18,6));

### Rolling Average smoothing

Various patterns can be seen on this daily line graph, but overal patterns may be hard to seen due to noise plentiful short spikes. In order to see a *smoother* patterns over time, a __rolling average__ can be applied to a time series. It walks over the timestamps with a given window (7 days for example) and calculates averages for each. 

Pandas can perform this via the `.rolling()` method which can be called on both a dataframe as well as a series object. The window size for this method can be set using both a fixed number of data points as well as particular time intervals (days, weeks etc).

In [ ]:
(
    power_daily2008
    .assign(rolling_mean=lambda df: df['consumption'].rolling('20D').mean())
    .plot(figsize=(16, 4))
);

Note that the orange *rolling mean line* is lagging behind what actually happens. This is because each point of it demonstrates information about all previous 20 days and not just about this particular moment. We can remove this effect using *centering*.

To center the rolling mean, we can either manually shift it backwards or use the option `center=True` for the `rolling()` method. You can see below how both achieve the same result - the red and green (overlapping) lines do not lag anymore. 

__Importantly__, centering requires information from the future for each point. This makes centering a "naughty" practice if we want to further make predictions about the future — this information is then already contained in the present data points, which is referred to as *data leakage* in machine learning.

In [ ]:
ax = (
    power_daily2008
    .assign(
        rolling_mean_d=lambda df: df['consumption'].rolling('20D').mean(),
        # ! With `center=True` window size cannot be a time frame
        rolling_mean_center=lambda df: df['consumption'].rolling(20, center=True).mean(),
        manual_center=lambda df: df['rolling_mean_d'].shift(-10)
    )
    .plot(figsize=(16,4))
)

ax.set_title('rolling_mean_center and manual_center overlap');

Rolling average smoothing is a simple way to isolate signal from noise in time series data and get an idea about general time series behavior.

It however has some important drawbacks:

- small window sizes can lead to more noise than signal
- it always lags by the window size (unless centered)
- it is not really informative about the future
- can be significantly skewed by extreme datapoints in the past

### BONUS: Exponential Smoothing

An alternative to calculating the rolling statistics is to smooth the timeseries exponentially with the following formula:

$$\hat{y_t} = \alpha y_t + (1-\alpha) \hat{y}_{t-1}$$

where $\hat{y_t}$ is the output of the exponential smoothing at time $t$, $y_t$ is the data point at time $t$, and $0<\alpha<1$ is the *the smoothing factor*.


The idea is to recursively smooth the series by averaging the current average with the current value. If $\alpha$ is high then the smoothing will be low but the average can respond quicker to changes, and if it is low — the result will be much more smooth and flat.

In [ ]:
(
    power_daily2008
    .assign(
        smoothed_01=lambda df: df['consumption'].ewm(alpha=0.1).mean(),
        smoothed_001=lambda df: df['consumption'].ewm(alpha=0.01).mean()
    )
    .plot(figsize=(16, 4))
);

Exponential Smoothing exhibits reduced lagging and more weight assigned to the current timestamps compared to Simple Rolling Averages, which also makes it more informative about the future.See more about ewm (exponentially weighted function) on the pandas [docs].

[docs]: https://pandas.pydata.org/pandas-docs/stable/user_guide/computation.html#exponentially-weighted-windows

Other alternatives are [weighted smoothing](http://pandas.pydata.org/pandas-docs/stable/user_guide/computation.html#rolling-windows) and [expanding windows](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.expanding.html). 

<a id='sum'></a>
## 5. Summary

We have covered: 
- Timestamps and formatting in Pandas
- How to properly read in time series data in Pandas, and why it is important to set the date as an index
- Time based manipulations, such as easy aggregations with `resample`, time-based features based on dtype `datetime` and shifting
- Rolling average smoothing, its disadvantages and alternatives

We should now be able to answer analytics questions like: 
- During what five day period was the most energy consumed?
- Which ten day period had the highest decrease in energy consumption between the first and the last day?
- Which week day has the highest power consumption on average? 
- Does the energy consumption drop in the weekend? 
- Could we replace missing values with smoothing? 
- Can we detect the holiday periods, based on the assumption that the average power consumption drops significantly for a few days during the holiday period? 
